In [ ]:
try:
    import tinygp
except ImportError:
    %pip install -q tinygp

try:
    import jaxopt
except ImportError:
    %pip install -q jaxopt

(quasisep-custom)=

# Custom Quasiseparable Kernels

````{admonition} Warning
:class: warning

This implementation of quasiseparable kernels is still experimental, and the models in this tutorial depend on low-level features that are subject to change.
````

The quasiseparable kernels built in to `tinygp` are all designed to be used with one-dimensional data (see {ref}`api-kernels-quasisep`), but one of the key selling points of the `tinygp` implementation over other similar projects (e.g. [celerite](https://celerite.readthedocs.io), [celerite2](https://celerite2.readthedocs.io), [S+LEAF](https://obswww.unige.ch/~delisle/spleaf/doc/)), is that it has a model building interface that is more expressive and flexible.
In this tutorial, we present some examples of the kinds of extensions that are possible within this framework.
This will be one of the most technical `tinygp` tutorials, and the implementation details are likely to change in future versions; you have been warned!

## Multivariate quasiseparable kernels

[Gordon et al. (2020)](https://arxiv.org/abs/2007.05799) demonstrated how the `celerite` algorithm could be extended to model "rectangular" data (e.g. parallel time series), and here we'll implement a slightly more general model that includes the Gordon et al. (2020) model as a special case.
But to start, let's implement something that is very similar to the simplest model from Gordon et al. (2020).
In this model, we have a single underlying Gaussian process, and each data point is generated from that process with a different amplitude.
To add a little more structure to the model, we'll imagine that we're modeling "multi-band" data where each observation is indexed by "time" (or some other one-dimensional input coordinate) and it's band ID (an integer).

We're not going to go into the mathematical details here (stay tuned for more details, or maybe even a publication?), but the methods that our custom kernel needs to overload here are {func}`tinygp.kernels.quasisep.Quasisep.coord_to_sortable` and {func}`tinygp.kernels.quasisep.Quasisep.observation_model`.
The first method (`coord_to_sortable`) takes our structured input (in this case a tuple with time as the first element, and band ID as the second), and returns a scalar that is sorted in the dataset (in this case, the time coordinate is what we need).
The second method (`observation_model`) is where the magic happens.
To get the behavior that we want here, we overload the `observation_model` by scaling the observation for each data point by the amplitude in that band.

Here's how we would implement this in `tinygp`:

In [ ]:
import jax
import jax.numpy as jnp

import tinygp

jax.config.update("jax_enable_x64", True)


@tinygp.helpers.dataclass
class Multiband(tinygp.kernels.quasisep.Wrapper):
    amplitudes: jnp.ndarray

    def coord_to_sortable(self, X):
        return X[0]

    def observation_model(self, X):
        return self.amplitudes[X[1]] * self.kernel.observation_model(X[0])

Some notes here:

1. We're using {class}`tinygp.kernels.quasisep.Wrapper` as our base class (instead of {class}`tinygp.kernels.quasisep.Quasisep`), since it provides some help when writing a custom kernel that wraps another quasiseparable kernel.
2. We've decorated our class with the `@tinygp.helpers.dataclass` decorator which, while not strictly necessary, can make our lives a little easier.

Now that we have this implementation, let's build an example model with a {class}`tinygp.kernels.quasisep.Matern52` as our base kernel and 3 bands.
Then we'll sample from it to get a sense for what is going on:

In [ ]:
def plot_multiband_sample(kernel):
    gp = tinygp.GaussianProcess(kernel, X)
    y = gp.sample(jax.random.PRNGKey(849))
    for i in np.unique(band_id):
        plt.axhline(-7 * i, color="k")
        plt.plot(
            t[band_id == i], y[band_id == i] - 7 * i, label=f"band {i + 1}"
        )
    plt.xlim(0, 10)
    plt.ylim(-21, 7)
    plt.yticks([])
    plt.xlabel("x")
    plt.ylabel("y + offset")
    plt.legend(fontsize=10, loc=2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

random = np.random.default_rng(394)
t = np.sort(random.uniform(0, 10, 700))
band_id = random.choice([0, 1, 2], size=len(t))
X = (t, band_id)

kernel = Multiband(
    kernel=tinygp.kernels.quasisep.Matern52(scale=1.5),
    amplitudes=jnp.array([3.1, -1.1, 3.7]),
)

plot_multiband_sample(kernel)

This model is very similar to the baseline model introduced by [Gordon et al. (2020; their Equation 13)](https://arxiv.org/abs/2007.05799), with the added generalization that we're not restricted to rectangular data: the observations don't need to be simultaneous.
While Gordon et al. (2020) showed that this model could be useful, it's not actually very expressive, so let's take another step.

The most obvious generalization of this simple model is to take the sum of several `Multiband` kernels with different amplitudes and, optionally, different underlying processes.
As an example, here's what happens if we take the sum of two of our custom kernels:

In [ ]:
kernel = Multiband(
    kernel=tinygp.kernels.quasisep.Matern52(scale=1.5),
    amplitudes=jnp.array([0.9, 0.7, -1.1]),
)
kernel += Multiband(
    kernel=tinygp.kernels.quasisep.Matern52(scale=0.5),
    amplitudes=jnp.array([1.1, -1.7, 1.5]),
)
plot_multiband_sample(kernel)

This is already a much more expressive kernel, with some shared structure between the two bands, but this relationship is much less restrictive.

We can also reproduce the full-rank Kronecker model from [Gordon et al. (2020)](https://arxiv.org/abs/2007.05799) using this same infrastructure.
In that case, we need to sum the same number of `Multiband` kernels as there are bands, using the same baseline kernel for each.
Then, if we call the $N_\mathrm{band} \times N_\mathrm{band}$ cross-band covariance matrix $R$ following Gordon et al. (2020), and take its Cholesky factorization $R = L\,L^\mathrm{T}$, the amplitude for the $n$-th term is the $n$-th row of $L$.
For example, we could use an exponential-squared kernel for the cross-band band covariance:

In [ ]:
R = 1.5**2 * jnp.exp(
    -0.5 * (jnp.arange(3)[:, None] - jnp.arange(3)[None, :]) ** 2
)
L = jnp.linalg.cholesky(R)

base_kernel = tinygp.kernels.quasisep.Matern52(scale=1.5)
kernel = sum(Multiband(kernel=base_kernel, amplitudes=row) for row in L)
plot_multiband_sample(kernel)